# Notebook to analyze and display scRNAseq data


### Load important modules

In [1]:
# Standard modules
import numpy as np
import os
import pandas as pd 
import plotly.graph_objects as go

# Move to root directory for easier module handling
os.chdir("../..")
print(os.listdir("."))

#LBAE imports
from modules.maldi_data import MaldiData
from modules.figures import Figures
from modules.atlas import Atlas
from modules.launch import Launch
from modules.storage import Storage


# multithreading/multiprocessing
from multiprocessing import Pool
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


['.git', '.gitattributes', '.vscode', 'LICENSE.md', 'README.md', 'TODO.py', 'app.py', 'assets', 'config.py', 'data_sample', 'documentation', 'index.py', 'js', 'main.py', 'modules', 'notebooks', 'pages', 'readme', 'requirements.txt', 'style', 'data', '__pycache__', '.gitignore']


#### Load LBAE objects

In [ ]:
path_data = "data/whole_dataset/"
path_annotations = "data/annotations/"
path_db = "data/app_data/data.db"

# Load shelve database
storage = Storage(path_db)

# Load data
data = MaldiData(path_data, path_annotations)

# If True, only a small portions of the figures are precomputed (if precomputation has not already
# been done). Used for debugging purposes.
sample = False

# Load Atlas and Figures objects. At first launch, many objects will be precomputed and shelved in
# the classes Atlas and Figures.
atlas = Atlas(data, storage, resolution=25)
figures = Figures(data, storage, atlas)

### Load data

In [ ]:
expr_table = pd.read_csv('notebooks/scRNAseq/data/expr_normalized_table.tsv', sep='\t')
meta_table = pd.read_csv('notebooks/scRNAseq/data/meta_table.tsv', sep='\t', usecols=[0, 2,3,4] )

In [ ]:
meta_table

In [ ]:
# Convert coordinates from metatable
meta_table["stereo_ML"] = (5.7 - meta_table["stereo_ML"])
meta_table["stereo_DV"] = (0.05 - meta_table["stereo_DV"])
meta_table["stereo_AP"] = (5.2 - meta_table["stereo_AP"])

# Display all dots in metatable
meta_table


In [ ]:
scatter = go.Scatter3d(
    x=(meta_table['stereo_AP'].to_numpy()[::1]-0.5)*1.2,
    y=(meta_table['stereo_ML'].to_numpy()[::1]-0)*1.2,
    z=(meta_table['stereo_DV'].to_numpy()+1)[::1]*1.2,
    mode='markers',
    marker=dict(
        size=1,
        opacity=0.8
    )
)

#fig = go.Figure(data=scatter)

#fig.show()

In [ ]:
# Get root figure
root_data = figures._storage.return_shelved_object(
    "figures/3D_page",
    "volume_root",
    force_update=False,
    compute_function=figures.compute_3D_root_volume,
)

In [ ]:
fig = go.Figure(data=[root_data, scatter])


# Hide grey background
fig.update_layout(
    margin=dict(t=0, r=0, b=0, l=0),
    scene=dict(
        xaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
        yaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
        zaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
    ),
)

# Set background color to zero
fig.layout.template = "plotly_dark"
fig.layout.plot_bgcolor = "rgba(0,0,0,0)"
fig.layout.paper_bgcolor = "rgba(0,0,0,0)"


fig.show()